# Convnet trained on both whitenoise and natural scenes for all cells in 15-10-07 experiment

In [1]:
import numpy as np
from os.path import expanduser
import os
import json
import theano
import tableprint
import pyret.filtertools as ft
import pyret.visualizations as pyviz
from keras.models import model_from_json
import h5py
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
%matplotlib inline
#%matplotlib qt

from pylab import rcParams
rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
rcParams['image.interpolation'] = 'nearest'
rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

Using gpu device 0: GeForce GT 750M (CNMeM is disabled)


In [7]:
# deep retina imports
import deepretina.visualizations as viz
from deepretina.preprocessing import datagen, loadexpt
from deepretina.modeltools import load_model, load_partial_model, list_layers, get_performance, get_weights, get_test_responses

## Load model

In [3]:
model_path = '/Users/lmcintosh/data/deep-retina/saved/lenna.salamander/2016-01-26 10.35.16 convnet/'
weight_name = 'epoch029_iter04250_weights.h5'

In [4]:
mdl = load_model(model_path, weight_name)

In [5]:
list_layers(model_path, weight_name)

+-----------+------------+-----------+
|layer      | weights    | biases    |
+-----------+------------+-----------+
|   layer_0 |    param_0 |    param_1|
|   layer_1 |            |           |
|   layer_2 |            |           |
|   layer_3 |            |           |
|   layer_4 |    param_0 |    param_1|
|   layer_5 |            |           |
|   layer_6 |    param_0 |    param_1|
+-----------+------------+-----------+


In [ ]:
true_white, predicted_white = get_test_responses(mdl, stim_type='white', cells=[0,1,2,3,4])
true_natural, predicted_natural = get_test_responses(mdl, stim_type='natural', cells=[0,1,2,3,4])

Loading testing data... 